In [1]:
import findspark
findspark.init()
import pyspark
import random
import os

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext()
spark = SparkSession(sc)

In [3]:
img_dir = "D:\\Data\\AnimalsOnTheWeb\\monkey"

In [4]:
imgs = spark.read.format("image").load(img_dir)

In [6]:
#imgs.show()

In [7]:
imgs.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [8]:
import pyspark.sql.types as typ

In [9]:
res_lbp = spark.read.csv('Res_LBP.csv',header=True)
labels =[
    ('ind',typ.IntegerType()),
    ('Animal',typ.StringType()),
    ('File',typ.StringType()),
    ('ID',typ.StringType()),
    ('LBP0',typ.FloatType()),
    ('LBP1',typ.FloatType()),
    ('LBP2',typ.FloatType()),
    ('LBP3',typ.FloatType()),
    ('LBP4',typ.FloatType()),
    ('LBP5',typ.FloatType()),
    ('LBP6',typ.FloatType()),
    ('LBP7',typ.FloatType()),
    ('LBP8',typ.FloatType()),
    ('LBP9',typ.FloatType()),
]
schema = typ.StructType([
    typ.StructField(e[0],e[1],False) for e in labels
])

In [10]:
res_lbp = spark.read.csv('Res_LBP.csv',header=True,schema=schema)
res_lbp.printSchema()

root
 |-- ind: integer (nullable = true)
 |-- Animal: string (nullable = true)
 |-- File: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- LBP0: float (nullable = true)
 |-- LBP1: float (nullable = true)
 |-- LBP2: float (nullable = true)
 |-- LBP3: float (nullable = true)
 |-- LBP4: float (nullable = true)
 |-- LBP5: float (nullable = true)
 |-- LBP6: float (nullable = true)
 |-- LBP7: float (nullable = true)
 |-- LBP8: float (nullable = true)
 |-- LBP9: float (nullable = true)



In [11]:
# include only records with these scores in list l
res_lbp = res_lbp.where(res_lbp.Animal.isin('alligator'))
# expected: (1,10), (1,20), (3,18), (3,18), (3,18)

In [12]:
res_lbp.count()

1445

In [13]:
import scipy.io as sio
file_path = 'D:\\Data\\AnimalsOnTheWeb\\alligator\\'
file = 'animaldata_alligator.mat'
data_read = sio.loadmat(os.path.join(file_path,file))

In [14]:
import numpy as np
import pandas as pd
import re

In [15]:
#aa = truth_name

In [16]:
truth_tbl = list(data_read['gt'][0])
truth_nameread = list(data_read['imgnames'][0])
truth_name = [t[0] for t in truth_nameread]

In [17]:
truth_lists = pd.DataFrame({'name': truth_name,'truth': truth_tbl})
#truth_lists = pd.DataFrame({'truth': truth_tbl})
truth_lists['name'] = truth_lists['name'].astype('str')

In [99]:
re_picid = re.compile('pic\d+')
aa = re_picid.findall(truth_lists['name'][0])
truth_lists['ID'] = [re_picid.findall(r)[0] for r in truth_lists['name']]

In [100]:
truth_lists.head()

,name,truth,ID
0,alligator/pic1_low_left.jpg,0,pic1
1,alligator/pic2_low_right.jpg,0,pic2
2,alligator/pic3_csm-amis.gif,0,pic3
3,alligator/pic4_csm-asin.gif,0,pic4
4,alligator/pic5_greenal.gif,0,pic5


In [101]:
df_truth = spark.createDataFrame(truth_lists)#,schema=schema1)
df_truth.printSchema()
df_truth.head(5)

root
 |-- name: string (nullable = true)
 |-- truth: long (nullable = true)
 |-- ID: string (nullable = true)



[Row(name='alligator/pic1_low_left.jpg', truth=0, ID='pic1'),
 Row(name='alligator/pic2_low_right.jpg', truth=0, ID='pic2'),
 Row(name='alligator/pic3_csm-amis.gif', truth=0, ID='pic3'),
 Row(name='alligator/pic4_csm-asin.gif', truth=0, ID='pic4'),
 Row(name='alligator/pic5_greenal.gif', truth=0, ID='pic5')]

In [103]:
df_truth = df_truth.withColumn('truth_int',df_truth['truth'].cast(typ.IntegerType()))
df_truth.printSchema()

root
 |-- name: string (nullable = true)
 |-- truth: long (nullable = true)
 |-- ID: string (nullable = true)
 |-- truth_int: integer (nullable = true)



In [47]:
#aa = df_truth.filter(df_truth.name.rlike('pic\d+_'))
#aa.head(15)

In [48]:
#split_col = pyspark.sql.functions.split(df_truth['name'], '\d')
#df_truth = df_truth.withColumn('NAME1', split_col.getItem(0))

In [49]:
df_ml = df_truth.join(res_lbp,on='ID')
df_ml

DataFrame[ID: string, name: string, truth: bigint, truth_int: int, ind: int, Animal: string, File: string, LBP0: float, LBP1: float, LBP2: float, LBP3: float, LBP4: float, LBP5: float, LBP6: float, LBP7: float, LBP8: float, LBP9: float]

In [50]:
df_ml.head(1)

[Row(ID='pic1', name='alligator/pic1_low_left.jpg', truth=0, truth_int=0, ind=556, Animal='alligator', File='pic1_low_left.jpg', LBP0=0.08529064059257507, LBP1=0.0923176109790802, LBP2=0.034361232072114944, LBP3=0.07235414534807205, LBP4=0.1361340880393982, LBP5=0.08859997987747192, LBP6=0.032899968326091766, LBP7=0.09792628884315491, LBP8=0.12302567809820175, LBP9=0.2370903640985489)]

In [51]:
df_ml1 = df_ml.select([c for c in df_ml.columns if c in ['truth_int','LBP0','LBP1','LBP2','LBP3','LBP4','LBP5','LBP6','LBP7','LBP8','LBP9']])

In [97]:
df_truth.describe()

DataFrame[summary: string, name: string, truth: string, ID: string, truth_int: string]

In [52]:
import pyspark.ml.feature as ft

In [72]:
labels1 =[
    ('truth_int',typ.IntegerType()),
    ('LBP0',typ.FloatType()),
    ('LBP1',typ.FloatType()),
    ('LBP2',typ.FloatType()),
    ('LBP3',typ.FloatType()),
    ('LBP4',typ.FloatType()),
    ('LBP5',typ.FloatType()),
    ('LBP6',typ.FloatType()),
    ('LBP7',typ.FloatType()),
    ('LBP8',typ.FloatType()),
    ('LBP9',typ.FloatType()),
]

In [73]:
featuresCreator = ft.VectorAssembler(
    inputCols=[col[0] for col in labels1[0:]],outputCol='features'
)

In [74]:
import pyspark.ml.classification as cl

In [75]:
logistic = cl.LogisticRegression(maxIter=10,regParam=0.01,labelCol='truth_int')

In [76]:
from pyspark.ml import Pipeline

In [77]:
pipeline = Pipeline(stages=[featuresCreator,logistic])

In [78]:
lbp_train, lbp_test = df_ml1.randomSplit([0.7,0.3],seed=1)

In [79]:
model = pipeline.fit(lbp_train)

In [84]:
test_model = model.transform(lbp_test)
test_model.take(1)

[Row(truth_int=0, LBP0=1.4100000043981709e-05, LBP1=0.0004844740033149719, LBP2=0.0002408639993518591, LBP3=0.004776970017701387, LBP4=0.007476892787963152, LBP5=0.032398831099271774, LBP6=0.007372186053544283, LBP7=0.014115380123257637, LBP8=0.9286929965019226, LBP9=0.004427279811352491, features=DenseVector([0.0, 0.0, 0.0005, 0.0002, 0.0048, 0.0075, 0.0324, 0.0074, 0.0141, 0.9287, 0.0044]), rawPrediction=DenseVector([4.1628, -4.1628]), probability=DenseVector([0.9847, 0.0153]), prediction=0.0)]

In [88]:
import pyspark.ml.evaluation as ev

In [91]:
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability',labelCol='truth_int')

In [92]:
print(evaluator.evaluate(test_model, {evaluator.metricName:'areaUnderROC'}))
print(evaluator.evaluate(test_model,{evaluator.metricName:'areaUnderPR'}))

1.0
1.0


In [94]:
lbp_train.describe()

DataFrame[summary: string, truth_int: string, LBP0: string, LBP1: string, LBP2: string, LBP3: string, LBP4: string, LBP5: string, LBP6: string, LBP7: string, LBP8: string, LBP9: string]